Инструкция:

Шаг 1.  Изучите материалы лекционных и практических занятий по теме 9.3.

Шаг 2. Постройте рекомендательную систему (Алгоритм выполнения представлен в лекции по теме 9.3).

Шаг 3. Реализуйте коллаборативную фильтрацию данных по заданным параметрам (Алгоритм выполнения представлен в лекции по теме 9.3).

Шаг 4. Опубликуйте ссылку на файл расширения ipynb, прикрепив его в этом задании.

**Примечание**: Здесь нужно построить рекомендации для любого, выбранного Вами, покупателя.

Матрица предпочтений представлена в файле `matrix_utility.*csv*`  

Реализовать рекомендации по двум подходам: user-based и item-based.

Меру сходства взять любую (корреляция Пирсона, косинусная мера, метрики расстояния).

Алгоритм:

Матрица предпочтений:

$$
I = \begin{array}{cccccc}
& Товар_1 & Товар_2 & \ldots & Товар_n \\
Пользователь_1 & r_{11} & r_{12} & \ldots & r_{1n}\\
Пользователь_2 & r_{21} & r_{22} & \ldots & r_{2n}\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
Пользователь_m & r_{m1} & r_{m2} & \ldots & r_{mn}\\
\end{array}
$$

Если для двух пользователей заданы их вектора предпочтений, то их сходство можно вычислить через коэффициент корреляции Пирсона
$$
w_{uv} = \dfrac{\sum_{i\in I_{uv}} (r_{ui}- \bar{r}_u)(r_{vi}- \bar{r}_v)}{
\sqrt{\sum_{i\in I_{uv}} (r_{ui}- \bar{r}_u)^2}\sqrt{\sum_{i\in I_{uv}} (r_{vi}- \bar{r}_v)^2}}
$$

Аналогично можно посчитать сходство двух товаров -- $w_{ij}$. 

Тогда рекомендации могут быть построены следующим образом:
- user-based:
$$
\hat{r}_{ui} = \bar{r}_u + \dfrac{\sum_{v\in U_{i}} sim(u,v)(r_{vi}-\bar{r}_v)}{\sum_{v\in U_{i}} sim(u,v)} 
$$

- item-based:
$$
\hat{r}_{ui} = \bar{r}_i + \dfrac{\sum_{j\in I_{u}} sim(i,j)(r_{uj}-\bar{r}_j)}{\sum_{j\in I_{u}} sim(i,j)} 
$$





In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('matrix_utility.csv')
df.describe()

,Товар_1,Товар_2,Товар_3,Товар_4,Товар_5,Товар_6,Товар_7,Товар_8,Товар_9,Товар_10,Товар_11,Товар_12,Товар_13,Товар_14,Товар_15,Товар_16,Товар_17,Товар_18,Товар_19,Товар_20
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,8.111111,2.333333,3.444444,5.777778,6.111111,9.000000,6.000000,1.222222,5.111111,1.777778,4.111111,3.666667,5.111111,5.000000,8.555556,4.000000,3.777778,6.888889,4.777778,4.222222
std,5.441609,4.153312,3.503966,7.259094,7.557189,4.974937,4.609772,2.538591,2.315407,2.438123,4.675587,3.840573,5.254628,4.301163,5.659309,3.937004,3.231787,5.925463,4.893306,3.833333
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,0.000000,0.000000,0.000000,0.000000,7.000000,3.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,2.000000,5.000000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,7.000000,0.000000,3.000000,4.000000,0.000000,9.000000,5.000000,0.000000,5.000000,1.000000,3.000000,3.000000,2.000000,5.000000,6.000000,3.000000,4.000000,8.000000,3.000000,3.000000
75%,13.000000,1.000000,7.000000,8.000000,12.000000,11.000000,8.000000,0.000000,5.000000,3.000000,6.000000,6.000000,9.000000,7.000000,14.000000,7.000000,5.000000,11.000000,9.000000,5.000000
max,16.000000,11.000000,9.000000,22.000000,18.000000,17.000000,14.000000,7.000000,10.000000,7.000000,12.000000,10.000000,14.000000,11.000000,15.000000,10.000000,9.000000,14.000000,11.000000,13.000000


Функция для подсчета косинусного расстояния между векторами. 

In [ ]:
def sim(vecA, vecB):
    res = (vecA*vecB).sum()/(vecA*vecA).sum()/(vecB*vecB).sum()
    return res

Начнём с рекомендации товара по пользователям (user-based)

Функция для рекомендации товара на основе всех пользователей

In [ ]:
def rec(dataframe, user):
    avg = dataframe.mean() #берём средние по столбцам
    i=0 #счётчик для строк
    df_user_0_c = dataframe-avg #датафрейм с удалением средних
    df_user_0 = dataframe #исходный датафрейм
    while i<(len(dataframe)): #проходим по строкам и формируем 2 новых датафрейма
        df_user_0_c.iloc[i] = df_user_0_c.iloc[i]*(sim(dataframe.iloc[user].values, dataframe.iloc[i].values))
        df_user_0.iloc[i] = df_user_0.iloc[i]*(sim(dataframe.iloc[user].values, dataframe.iloc[i].values))
        i+=1
        if 'Товар_' in str(dataframe.columns[1]):
            s1 = df_user_0_c.drop(user).sum() #суммы числителей (без выбранного пользователя)
            s2 = df_user_0.drop(user).sum() #суммы знаменателей (без выбранного пользователя)
        else:
            s1 = df_user_0_c.drop('Товар_'+str(user+1)).sum() #суммы числителей (без выбранного пользователя)
            s2 = df_user_0.drop('Товар_'+str(user+1)).sum() #суммы знаменателей (без выбранного пользователя)
    coefs = s1/s2 #общие коэффициенты по товарам
    i=0 #очередной счётчик для цикла
    i_max = -100 #т.к. значения могут быть и отрицательными ставим -100
    recomendation = ''
    print('Новые для пользователя товары:')
    for item in dataframe: #проходим по исходному датафрейму в поисках нулей
        if dataframe[item].iloc[user]==0: #находим максимальный коэффициент для нулевых значений
            if 'Товар_' in str(dataframe.columns[1]):
                print(f'Название: {item}, коэффициент {coefs[int(item.replace("Товар_",""))-1]}')
                if i_max <= coefs[int(item.replace('Товар_',''))-1]: #-1 т.к. нумерация с нуля
                    i_max = coefs[int(item.replace('Товар_',''))-1]
                    recomendation = item 
            else:                
                print(f'Название: {item}, коэффициент {coefs[int(item)]}')
                if i_max <= coefs[int(item)]: 
                    i_max = coefs[int(item)]
                    recomendation = item 
    print (f'Рекомендация - {str(recomendation)} с коэффициентом  {i_max}')

Функция для рекомендации товара на основе наиболее близкого элемента

In [ ]:
def rec_target(dataframe, user):
    avg = dataframe.mean() #берём средние по столбцам
    i=0 #счётчик для строк
    df_user_0_c = dataframe-avg #датафрем с удалением средних
    df_user_0 = dataframe #исходный датафрейм 
    cos_max = -100
    cos_user = user
    while i<(len(dataframe)): #проходим по строкам в поисках наиболее похожего пользователя
        if i!= user:
            sm = sim(dataframe.iloc[user].values, dataframe.iloc[i].values)            
            if cos_max < sm:
                cos_max = sm
                cos_user = i
        i+=1
    if 'Товар_' in str(dataframe.columns[1]):
        print(f'Наиболее подходящий пользователь - {cos_user}')
    else:
        print(f'Наиболее подходящий элемент - {cos_user}')
    s1 = df_user_0_c.iloc[cos_user]*(sim(dataframe.iloc[user].values, dataframe.iloc[cos_user].values)) #суммы числителей
    s2 = df_user_0.iloc[cos_user]*(sim(dataframe.iloc[user].values, dataframe.iloc[cos_user].values)) #суммы знаменателей
    coefs = s1/s2 #общие коэффициенты по товарам
    coefs = coefs.replace(-inf,-100)
    i=0 #очередной счётчик для цикла
    i_max = -100 #т.к. значения могут быть и отрицательными ставим -100
    recomendation = ''
    print('Новые для пользователя товары:')
    for item in dataframe: #проходим по исходному датафрейму в поисках нулей
        if dataframe[item].iloc[user]==0: #находим максимальный коэффициент для нулевых значений
            if 'Товар_' in str(dataframe.columns[1]):
                print(f'Название: {item}, коэффициент {coefs[int(item.replace("Товар_",""))-1]}')
                if i_max <= coefs[int(item.replace('Товар_',''))-1]: #-1 т.к. нумерация с нуля
                    i_max = coefs[int(item.replace('Товар_',''))-1]
                    recomendation = item 
            else:                
                print(f'Название: {item}, коэффициент {coefs[int(item)]}')
                if i_max <= coefs[int(item)]:
                    i_max = coefs[int(item)]
                    recomendation = item 
    print (f'Рекомендация - {str(recomendation)} с коэффициентом  {i_max}')

In [ ]:
rec(df, 2)

Новые для пользователя товары:
Название: Товар_2, коэффициент -0.6189013525316431
Название: Товар_3, коэффициент 0.1751594201304232
Название: Товар_4, коэффициент -0.17930610490110765
Название: Товар_8, коэффициент 0.19673168743531627
Название: Товар_13, коэффициент 0.05714657493942517
Название: Товар_16, коэффициент -0.08088222801633693
Название: Товар_17, коэффициент 0.016116081281377793
Название: Товар_18, коэффициент -0.06655875251785018
Рекомендация - Товар_8 с коэффициентом  0.19673168743531627


In [ ]:
rec_target(df, 2)

Наиболее подходящий пользователь - 0
Новые для пользователя товары:
Название: Товар_2, коэффициент -3176.9567928691276
Название: Товар_3, коэффициент -2300.869842235941
Название: Товар_4, коэффициент -490.02453780911287
Название: Товар_8, коэффициент -837.7279930490716
Название: Товар_13, коэффициент -11951.609419033923
Название: Товар_16, коэффициент -814.9681442611644
Название: Товар_17, коэффициент -1692.2228863199978
Название: Товар_18, коэффициент -1293.682565670838
Рекомендация -  с коэффициентом  -100


Т.к. функции рассчитаны на оба варианта использования, транспонируем матрицу и находим рекомендации по товарам

In [ ]:
df_t = df.T

In [ ]:
rec(df_t, 2)

Новые для пользователя товары:
Название: 2, коэффициент 0.02308135935255081
Название: 3, коэффициент -0.35294067813107755
Название: 7, коэффициент -0.07609329041322235
Рекомендация - 2 с коэффициентом  0.02308135935255081
